In [3]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd


# url = 'https://www.tripadvisor.com/Restaurant_Review-g186299-d11909730-Reviews-or10-Cosmo-Southampton_Hampshire_England.html'


def crawl_review_container(link, filename, pages):
    url = link
    pages = int(pages)
    visit_dates = []
    titles = []
    reviews = []
    ratings = []
    visit_ids = []
    for round in range(pages):
        res = requests.get(url)

        html_soup = BeautifulSoup(res.text, 'html.parser')

        # 開始爬資料
        review_containers = html_soup.find_all('div', class_='review-container')

        for container in review_containers:
            # date_of_visit
            visit_date = container.find('div', class_='prw_rup prw_reviews_stay_date_hsx')
            visit_date = visit_date.text[15:]
            visit_dates.append(visit_date)

            # titles
            title = container.find('span', attrs={'class': 'noQuotes'})
            title = title.text
            titles.append(title)

            # review
            review = container.find('p', class_='partial_entry')
            review = review.text
            reviews.append(review)

            # rating
            rating = container.find('div', attrs={'ui_column is-9'})
            rating = int(rating.span['class'][1][7:]) / 10
            ratings.append(rating)

            # ID
            visit_id = container.find('div', attrs={'class': 'memberOverlayLink clickable'})['id'][-9:]
            visit_id = int(visit_id)
            visit_ids.append(visit_id)

        paging_url = html_soup.select('div.mobile-more a')[15]['href']

        next_url = 'https://www.tripadvisor.com/' + paging_url

        print('success for crawler :', url)
        
        url = next_url

        time.sleep(20)
    # save to csv
    df = pd.DataFrame({'rating': ratings,
                       'review': reviews,
                       'title': titles,
                       'user_id': visit_ids,
                       'visit_date': visit_dates
                       })

    df.to_csv( filename +'.csv', encoding='utf-8')
    df = pd.read_csv(filename+'.csv')
    df.head()


crawl = input('insert the url：')
csv_name = input('insert the filename:')
pages = input('insert the number of pages:')
print('start crawling')
crawl_review_container(crawl, csv_name, pages)
print(csv_name + 'is already save to .csv')

insert the url：https://www.tripadvisor.com/Restaurant_Review-g186299-d16693790-Reviews-Tim_s_Bistro-Southampton_Hampshire_England.html
insert the filename:tt
insert the number of pages:2
start crawling
success for crawler : https://www.tripadvisor.com/Restaurant_Review-g186299-d16693790-Reviews-Tim_s_Bistro-Southampton_Hampshire_England.html
success for crawler : https://www.tripadvisor.com//Restaurant_Review-g186299-d16693790-Reviews-or10-Tim_s_Bistro-Southampton_Hampshire_England.html
ttis already save to .csv
